In [16]:
from pathlib import Path
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib as plt
import pursuit_functions
import pyarrow as pa
import re

pd.set_option('display.max_columns', 100)  # Show more columns (default is 20)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#load region files
data_dir = Path("/Users/may/Projects/nitzPurusitData")
#data_dir = Path("/Volumes/ASA_Lab/Data/Andy/nitzPurusitData")
#data_dir = Path("/Volumes/ASA_Lab/Data/Xiaoxiao/ppcRscEVCPoster/pursuitSessionFiles")


region_directories = pursuit_functions.file_reader.load_region_files(data_dir, prefix='sl')

Loading: /Users/may/Projects/nitzPurusitData/slPar.mat
Loading: /Users/may/Projects/nitzPurusitData/slRsc.mat


In [3]:
#set a variable to a region_directories file
rsc_directory = pd.DataFrame(region_directories['slRsc'])
ppc_directory = pd.DataFrame(region_directories['slPar']) 

In [4]:
rsc_directory.head()

,sessFile,tt,cellIndex,spkTimes,blocks
0,KB10_01_pursuitRoot.mat,24,1,"[0.054125, 0.200625, 0.772075, 1.2124, 1.44902...","[[nan, nan], [6152.0, 141262.0], [nan, nan]]"
1,KB10_02_pursuitRoot.mat,17,1,"[0.1669, 0.28305, 0.380875, 0.751525, 0.764025...","[[88504.0, 144672.0], [8320.0, 86109.0], [nan,..."
2,KB10_02_pursuitRoot.mat,25,2,"[0.137075, 1.542725, 1.817975, 1.9046, 1.9853,...","[[88504.0, 144672.0], [8320.0, 86109.0], [nan,..."
3,KB10_03_pursuitRoot.mat,17,1,"[0.31265, 0.40455, 0.445, 0.4775, 0.518525, 0....","[[96666.0, 150971.0], [7012.0, 95499.0], [nan,..."
4,KB10_04_pursuitRoot.mat,17,1,"[0.42885, 0.527525, 0.57495, 0.750225, 0.87707...","[[nan, nan], [8354.0, 119997.0], [nan, nan]]"


In [5]:
def unpack_block_indices(df):
    blocks = ["FE1", "pursuit", "FE2"]

    for i, blocks in enumerate(blocks):
        df[f"{blocks}_start"] = df["blocks"].apply(lambda x: x[i][0] if len(x) > i else None)
        df[f"{blocks}_end"]   = df["blocks"].apply(lambda x: x[i][1] if len(x) > i else None)
    
    return df


In [6]:
unpack_block_indices(rsc_directory)
unpack_block_indices(ppc_directory)

,sessFile,tt,cellIndex,spkTimes,blocks,FE1_start,FE1_end,pursuit_start,pursuit_end,FE2_start,FE2_end
0,KB09_12_pursuitRoot.mat,21,1,"[5.25295, 9.17145, 11.37925, 12.109775, 12.778...","[[nan, nan], [4559.0, 71201.0], [nan, nan]]",NaN,NaN,4559.0,71201.0,NaN,NaN
1,KB09_12_pursuitRoot.mat,21,2,"[119.3058, 138.72345, 228.91498, 350.39112, 35...","[[nan, nan], [4559.0, 71201.0], [nan, nan]]",NaN,NaN,4559.0,71201.0,NaN,NaN
2,KB09_12_pursuitRoot.mat,29,3,"[0.15415, 1.331125, 1.53245, 1.91845, 2.067575...","[[nan, nan], [4559.0, 71201.0], [nan, nan]]",NaN,NaN,4559.0,71201.0,NaN,NaN
3,KB09_13_pursuitRoot.mat,9,1,"[0.006275, 0.0101, 0.029525, 0.147625, 0.20227...","[[nan, nan], [14312.0, 71428.0], [nan, nan]]",NaN,NaN,14312.0,71428.0,NaN,NaN
4,KB09_13_pursuitRoot.mat,21,2,"[0.0736, 0.2598, 0.279675, 0.301375, 0.37855, ...","[[nan, nan], [14312.0, 71428.0], [nan, nan]]",NaN,NaN,14312.0,71428.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
362,KB20_21_pursuitRoot.mat,17,2,"[0.0489, 0.0708, 0.298175, 1.332725, 3.047975,...","[[95903.0, 166791.0], [5602.0, 90432.0], [nan,...",95903.0,166791.0,5602.0,90432.0,NaN,NaN
363,KB20_21_pursuitRoot.mat,17,3,"[0.03125, 0.052075, 0.05875, 0.0662, 0.072675,...","[[95903.0, 166791.0], [5602.0, 90432.0], [nan,...",95903.0,166791.0,5602.0,90432.0,NaN,NaN
364,KB20_21_pursuitRoot.mat,17,4,"[4.30265, 4.364875, 7.93885, 7.955475, 8.6423,...","[[95903.0, 166791.0], [5602.0, 90432.0], [nan,...",95903.0,166791.0,5602.0,90432.0,NaN,NaN
365,KB20_24_pursuitRoot.mat,17,1,"[0.18525, 0.2389, 0.300475, 0.438275, 0.63885,...","[[129274.0, 177655.0], [4468.0, 126200.0], [na...",129274.0,177655.0,4468.0,126200.0,NaN,NaN


In [8]:
#load pursuit session files
data_dir2 = Path("/Users/may/Projects/nitzPurusitData/Sessions")

rsc_files = rsc_directory["sessFile"].unique().tolist()
ppc_files = ppc_directory["sessFile"].unique().tolist()
include_files = list(set(rsc_files) | set(ppc_files))

session_files = pursuit_functions.file_reader.load_session_files(data_dir2, include_files=include_files)


Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB20_12_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB19_07_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB10_20_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/LP06_30_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/LP03_20_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB09_23_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/LP06_29_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/LP03_04_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/LP06_14_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB10_04_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB19_23_pursuitRoot.mat
Skipping: /Users/may/Projects/nitzPurusitData/Sessions/KB09_40_pursuitRoot.mat
Loading: /Users/may/Projects/nitzPurusitData/Sessions/KB20_04_p

In [11]:
#convert extracted pursuit session data into dataframes
def convert_sessions_to_dfs(df):
    session_dfs = {
        filename: pd.DataFrame(file_data).convert_dtypes(dtype_backend="pyarrow")
        for filename, file_data in pursuit_session_files.items()
    }
    return session_dfs 

In [12]:
session_dfs = convert_sessions_to_dfs(session_files)

In [13]:
session_dfs["KB10_01_pursuitRoot.mat"].head()

,time,ratPos_1,ratPos_2,laserPos_1,laserPos_2,ratVel,ratAcc_1,ratAcc_2,laserVel,laserAcc_1,laserAcc_2,ratMoveDir,laserMoveDir,ratHeadDir,ratAV,laserAV,spkTable_1
0,0.016667,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.479576,<NA>,<NA>,<NA>,<NA>,0
1,0.033334,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.58729,<NA>,<NA>,0.107714,<NA>,0
2,0.050001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.675603,<NA>,<NA>,0.088313,<NA>,1
3,0.066668,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.749041,<NA>,<NA>,0.073438,<NA>,0
4,0.083335,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.811625,<NA>,<NA>,0.062584,<NA>,0


In [22]:
def filter_spkTables(region_df, session_dfs):

    filtered_dfs = {}

    grouped = region_df.groupby("sessFile")["cellIndex"].apply(list) 

    for session, df in session_dfs.items():
        if session not in grouped:
            
            filtered_dfs[session] = df.copy() 
            
            continue
        
        valid_cells = [str(c) for c in grouped[session]]
        
        keep_spk_cols = []
        
        for col in df.columns:
            if col.startswith("spkTable"):
                for cell in valid_cells:
                    if re.fullmatch(rf"spkTable_?{cell}", col):
                        keep_spk_cols.append(col)
        
        
        non_spk_cols = [c for c in df.columns if not c.startswith("spkTable")]
        keep_cols = non_spk_cols + keep_spk_cols
        filtered_dfs[session] = df[keep_cols].copy()
    
    return filtered_dfs

In [23]:
rsc_session_dfs = filter_spkTables(rsc_directory, session_dfs)

In [24]:
rsc_session_dfs["KB10_09_pursuitRoot.mat"].head()

,time,ratPos_1,ratPos_2,laserPos_1,laserPos_2,ratVel,ratAcc_1,ratAcc_2,laserVel,laserAcc_1,laserAcc_2,ratMoveDir,laserMoveDir,ratHeadDir,ratAV,laserAV,spkTable_2,spkTable_3,spkTable_4
0,0.016667,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.614159,<NA>,<NA>,<NA>,<NA>,0,0,0
1,0.033334,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.679099,<NA>,<NA>,0.064939,<NA>,0,0,1
2,0.050001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.754666,<NA>,<NA>,0.075567,<NA>,0,0,0
3,0.066668,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.843714,<NA>,<NA>,0.089048,<NA>,0,0,0
4,0.083335,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.950729,<NA>,<NA>,0.107015,<NA>,0,0,1


In [ ]:
rsc_session_dfs["KB20_01_pursuitRoot.mat"].head()

In [27]:
def count_spkTables(session_dfs):
    grand_total = 0
    for session, df in session_dfs.items():
        spk_count = sum(col.startswith("spkTable") for col in df.columns)
        print(f"{session}: {spk_count} spkTable columns")
        grand_total += spk_count
    
    print(f"\nGrand total: {grand_total} spkTable columns across all sessions")


In [28]:
count_spkTables(rsc_session_dfs)

KB20_12_pursuitRoot.mat: 3 spkTable columns
KB19_07_pursuitRoot.mat: 13 spkTable columns
KB10_20_pursuitRoot.mat: 1 spkTable columns
LP06_30_pursuitRoot.mat: 6 spkTable columns
LP03_20_pursuitRoot.mat: 2 spkTable columns
KB09_23_pursuitRoot.mat: 15 spkTable columns
LP06_29_pursuitRoot.mat: 3 spkTable columns
LP03_04_pursuitRoot.mat: 11 spkTable columns
LP06_14_pursuitRoot.mat: 10 spkTable columns
KB10_04_pursuitRoot.mat: 2 spkTable columns
KB19_23_pursuitRoot.mat: 8 spkTable columns
KB20_04_pursuitRoot.mat: 2 spkTable columns
KB19_08_pursuitRoot.mat: 13 spkTable columns
KB19_35_pursuitRoot.mat: 4 spkTable columns
LP06_02_pursuitRoot.mat: 12 spkTable columns
LP03_12_pursuitRoot.mat: 2 spkTable columns
KB10_15_pursuitRoot.mat: 1 spkTable columns
LP06_05_pursuitRoot.mat: 3 spkTable columns
KB19_32_pursuitRoot.mat: 4 spkTable columns
KB20_03_pursuitRoot.mat: 4 spkTable columns
KB19_16_pursuitRoot.mat: 11 spkTable columns
KB19_19_pursuitRoot.mat: 10 spkTable columns
KB19_24_pursuitRoot.mat:

In [ ]:
rsc_session_dfs["KB10_01_pursuitRoot.mat"].head()

In [ ]:
def make_region_df(region_df, session_dfs, region: str):    
    region_dfs = {}

    dfs = []
    for session in region_df["sessFile"].unique():
        if session in session_dfs:
            df = session_dfs[session].copy()
            df['region'] = region
            dfs.append(df)

    if dfs:
        region_dfs[region] = pd.concat(dfs, ignore_index=True)
    
    return region_dfs   
